In [1]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import random

In [17]:
#get data from grammar's csv file of a single type format
cluster_info = pd.read_csv("test_files/s_exp_features_test.csv")
cluster_info.head()
#at what index is the s-expression id in this? once we know this, we can add the s-expression id to each node's lsit of s-expressions

,Unnamed: 0,0,1,2,3,4,5
0,0,2.0,0.125000,0.584635,4.0,0.0,0.0415
1,1,1.0,0.678385,-0.678385,0.0,0.0,0.4068
2,2,5.0,0.036458,-11.315104,1.5,1.0,1.5536
3,3,1.0,0.003906,-2.003906,0.0,0.0,0.0004
4,4,7.0,0.170573,-31.157552,4.0,2.0,1.5620


In [19]:
k_size = int(len(cluster_info)/4) #k_size is arbitrarily set, equal to the number of clusters
print(k_size)
kmeans = KMeans(n_clusters=k_size).fit(cluster_info)

23


In [20]:
#outputs which cluster number the data point is placed into
kmeans.labels_

array([13, 13, 13, 13,  3,  3,  3,  3, 22,  3, 22, 22,  3,  3, 22,  3, 20,
       11,  3, 22,  1, 20, 11, 11,  1,  1,  4, 11, 20,  1, 16, 11,  4, 16,
       14,  4, 14,  1,  4, 15, 16,  4, 15, 11, 15, 16,  4, 19, 14, 16, 14,
       19, 17, 17, 15, 17, 18, 21, 21,  8, 19, 19, 18, 17,  0,  6, 21,  7,
        6, 21, 12, 12,  5,  0, 12, 12,  6,  0, 12,  6,  7, 12,  8,  2, 10,
       10, 10,  2,  2,  2,  6,  9], dtype=int32)

In [5]:
#kmeans.predict(fill in with our own 7 unit vector data), will classify it into cluster

In [6]:
class Node:
    def __init__(self, node_num):
        self.node_num = node_num
        self.s_exp = [] #list of s-exp-ids
        self.cpt = {} #maps from node_num to conditional probability
        self.count = 0
    
    def add_exp(self, s):
        self.s_exp.append(s)

In [11]:
#dictionary that maps s-exp-id to: s-exp, song-id (which song), song-index (position in song)
s_exp_dict = {} 

#assume s_exp_labels is grammar output with 3 columns: s-exp, song-id, song-index
#s_exp_labels = np.loadtxt(open("test_files/test_exp.csv", "r"), dtype="str", delimiter=",", skiprows=1)
s_exp_labels = pd.read_csv('test_files/s_exp_test.csv')
s_exp_labels

,Unnamed: 0,exp,song_id,song_index
0,0,0 0,0,0
1,1,0 4 H0.155 R0.725,0,6
2,2,0 2 C0.340 H0.174,0,7
3,3,-3 1 C0.164 C0.195 H0.242 H0.079 R0.243,0,8
4,4,-3 7 C0.073 H0.130 C0.185 C0.091 C0.138 C0.089...,0,10
5,5,-2 2 C0.100 C0.138 R0.150 C0.147,0,11
6,6,-3 5 H0.070 H0.182 C0.164 R0.264,0,12
7,7,0 3 C0.082 C0.108 H0.077 H0.133,0,13
8,8,-6 12 H0.083 R0.118 R0.219 H0.086 C0.152 C0.08...,0,14
9,9,0 2 C0.168 C0.350,0,15


In [13]:
# for expression in s_exp_labels:
#     s_exp_dict[expression[0]] = [expression[1], expression[2], expression[3]]
for index, row in s_exp_labels.iterrows():
    s_exp_dict[index] = [row['exp'], row['song_id'], row['song_index']]

s_exp_dict

{0: ['0 0 ', 0, 0],
 1: ['0 4 H0.155 R0.725 ', 0, 6],
 2: ['0 2 C0.340 H0.174 ', 0, 7],
 3: ['-3 1 C0.164 C0.195 H0.242 H0.079 R0.243 ', 0, 8],
 4: ['-3 7 C0.073 H0.130 C0.185 C0.091 C0.138 C0.089 C0.333 ', 0, 10],
 5: ['-2 2 C0.100 C0.138 R0.150 C0.147 ', 0, 11],
 6: ['-3 5 H0.070 H0.182 C0.164 R0.264 ', 0, 12],
 7: ['0 3 C0.082 C0.108 H0.077 H0.133 ', 0, 13],
 8: ['-6 12 H0.083 R0.118 R0.219 H0.086 C0.152 C0.082 R0.190 ', 0, 14],
 9: ['0 2 C0.168 C0.350 ', 0, 15],
 10: ['0 3 R0.059 H0.060 C0.116 H0.096 C0.143 H0.090 R0.220 ', 0, 16],
 11: ['-1 3 H0.115 R0.306 C0.141 H0.158 R0.138 ', 0, 17],
 12: ['0 0 C0.150 C0.288 ', 0, 18],
 13: ['0 0 R0.184 R0.150 H0.475 ', 0, 19],
 14: ['-1 1 C0.107 C0.229 H0.076 C0.271 ', 0, 20],
 15: ['0 2 C0.073 H0.168 C0.056 ', 0, 21],
 16: ['-4 5 C0.148 C0.103 C0.074 C0.057 R0.047 C0.053 C0.158 C0.039 R0.168 H0.191 ',
  0,
  22],
 17: ['-10 9 C0.108 R0.145 H0.143 H0.236 ', 0, 23],
 18: ['0 0 H0.201 R0.249 ', 0, 24],
 19: ['0 0 R0.234 R0.201 H0.576 ', 0, 25],

In [14]:
node_objects = [] #list of nodes for the Markov chain

for cluster_num in range(k_size):
    node_objects.append(Node(cluster_num)) #initialize 1 node for each cluster number
    
for i in range(0,len(kmeans.labels_)): #iterating through all the data points
    cluster_num = kmeans.labels_[i] #access the cluster num each data point corresponds to
    #adding s-exp, song-id (which song), song-index (position in song) to the node object
    node_objects[cluster_num].add_exp(cluster_info[i][0]) #why string??? (previously)
    # find how to add the s-expression id correspondng to that node
    # index 0 is s-expression

In [15]:
#create cpt
for outer_node in node_objects:
    for inner_node in node_objects:
        outer_node.cpt[inner_node.node_num] = 0
        #if (outer_node != inner_node):
        for s_expression_outer in outer_node.s_exp:
            for s_expression_inner in inner_node.s_exp:
                #print(s_expression_outer)
                #print(s_exp_dict)
                if s_exp_dict[s_expression_outer][1] == s_exp_dict[s_expression_inner][1] and (s_exp_dict[s_expression_inner][2] - s_exp_dict[s_expression_outer][2] == 1):
                    outer_node.count += 1
                    outer_node.cpt[inner_node.node_num] += 1
    #creates the probability of going to the next node, not s-exp
    outer_node.cpt = {k: (v/outer_node.count) for k, v in outer_node.cpt.items() if outer_node.count != 0} 

In [21]:
def weighted_random_by_dct(dct):
   rand_val = random.random() #random value between 0 and 1
   total = 0
   for k, v in dct.items():
       total += v
       if rand_val <= total: #if running total exceeds probability, that's what you want
           return k

In [26]:
#takes in a number and returns list of s-exp ids generated by the cpt ordering given by probabilities
#cpt jumps from s-exp to s-exp
#nodes are collection of s-exp (s-exp is 1 measure with all those features)
def s_expressions(n): #n is length of 
    s_exp_ids = [] #create empty list
    start = int(random.uniform(0, len(node_objects))) #random start node
    next_node = node_objects[start] #get that node
    for i in range(n):
        next_node_num = weighted_random_by_dct(next_node.cpt) #get the next node based on the current node's cpt
        print(next_node_num)
        print(next_node.cpt)
        next_node = node_objects[next_node_num] #set the next node variable to that node
        next_s_exp_id = random.choice(next_node.s_exp) #get a random s-expression in that node 
        s_exp_ids += [next_s_exp_id] #store id in the list
    return s_exp_ids

In [29]:
node_objects[1].s_exp

[17.0, 22.0]

In [23]:
possible_notes = lambda a, b : ['G']
select_note = lambda a, b, c, d: 'C'

In [27]:
#go from s-exp to producing actual notes by calling possible_notes which passes into select_note
def produce_notes(num_of_measures, list_of_chords): #length of chords list = num of measures
    notes_df = pd.DataFrame(columns=["note", "start_time", "duration"])
    s_exp_ids = s_expressions(num_of_measures)
    for i in range(num_of_measures): #i refers to measure number
        s_exp = s_exp_dict[s_exp_ids[i]][0] #ith s-exp, which is a string
        split_list = s_exp.split(" ")
        curr_note = "Bb4" #figure out how to start the curr_note initially
        min_slope = split_list[0]
        max_slope = split_list[1]
        for j in range(2, len(split_list)):
            category = split_list[j].split("|")[0]
            start = split_list[j].split("|")[1]
            duration = split_list[j].split("|")[2]
            #possible_notes is looking at which other notes match note and category
            poss_notes_list = possible_notes(list_of_chords[i], category)
            selected_note = select_note(poss_notes_list, curr_note, min_slope, max_slope)
            notes_df.loc[j-2] = [selected_note, start+i, duration]
            curr_note = selected_note
    return notes_df
notes_df = produce_notes(2, ['Bb', 'A'])
notes_df

1
{0: 0.0, 1: 1.0}
None
{}


TypeError: list indices must be integers or slices, not NoneType